This work does not serve any academic or political purpose. It has been prepared solely using publicly available resources on the internet and is intended for educational purposes only.

Bu çalışma, herhangi bir akademik ya da siyasi amaç taşımamaktadır. Tamamen internetteki herkese açık kaynaklar kullanılarak hazırlanmış olup yalnızca eğitim amaçlıdır.

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sls

I import the libraries that we use in this project

In [2]:
candidates = pd.read_json("election_datas/CumhurbaskaniAdayListesi.json")
candidates["Adı Soyadı"] = candidates["Adı"]+" "+candidates["Soyadı"]
new_order = ["Adı Soyadı","Üniversite/Yüksekokul","Doğum Tarihi","Aday Sıra No"]
candidates = candidates.reindex(columns=new_order)

In [3]:
election_results_raw = pd.read_json("election_datas/SecimSonucIl.json")
election_results_raw.head(10)

,İl Id,İl Adı,Kayıtlı Seçmen Sayısı,Oy Kullanan Seçmen Sayısı,Geçerli Oy Toplamı,RECEP TAYYİP ERDOĞAN,MUHARREM İNCE,KEMAL KILIÇDAROĞLU,SİNAN OĞAN
0,,İLLER TOPLAMI,60.774.566,53.993.683,52.972.934,26.086.102,216.470,23.873.749,2.796.613
1,Oy Oranı,%49.24,%0.4,%45.06,%5.27,,,,
2,1,ADANA,1.615.326,1.414.720,1.388.925,610.132,5.110,706.780,66.903
3,Oy Oranı,%43.92,%0.36,%50.88,%4.81,,,,
4,2,ADIYAMAN,399.303,327.349,319.326,211.389,1.203,99.771,6.963
5,Oy Oranı,%66.19,%0.37,%31.24,%2.18,,,,
6,3,AFYONKARAHİSAR,536.910,484.078,472.801,301.941,2.642,136.507,31.711
7,Oy Oranı,%63.86,%0.55,%28.87,%6.7,,,,
8,4,AĞRI,307.114,223.749,215.851,68.468,873,142.021,4.489
9,Oy Oranı,%31.72,%0.4,%65.79,%2.07,,,,


I import the main datas but as you can see election_results_raw data is poorly labeled and a bir corrupted

In [4]:
from unidecode import unidecode
columns = ["Province Name"] + [f"{name} Percent" for name in candidates["Adı Soyadı"]]

election_result_percentages = pd.DataFrame(columns=columns)

for i in range(3,election_results_raw.shape[0],2):
    province = election_results_raw.iloc[i-1, 1]
    province = unidecode(province).capitalize()
    percentages = election_results_raw.iloc[i]
    new_row = [province]
    
    for j in range(1,5):
        candidate = candidates["Adı Soyadı"][j-1]
        percent_float = float(percentages.iloc[j].strip().strip("%"))
        new_row.append(percent_float)
        
    new_row = pd.DataFrame([new_row], columns=columns)
    election_result_percentages = pd.concat([election_result_percentages, new_row], ignore_index=True)
    
    percent_columns = [col for col in election_result_percentages.columns if "Percent" in col]
    election_result_percentages[percent_columns] = election_result_percentages[percent_columns].astype(float)

I clean and separate the data in the code below. The loop function checks every row and separates the election results by candidates.

In [5]:
avg_education = pd.read_excel("election_datas/illere gore ortalama egitim suresi.xltx")

avg_iq = pd.read_csv("election_datas/turkiye_mean_iq_data.csv")

I import average IQ levels and average education duration data by province. These two datasets may help us investigate the differences between the candidates and their voters.

In [6]:
avg_education["Province name"] = avg_education["Province name"].str.strip()

avg_iq.rename(columns={"#":"Province Name","İl":"Average IQ"},inplace=True)

avg_education.rename(columns={"Total":"Average Education Duration",
                              "Province name":"Province Name"},
                     inplace=True)


Some columns were causing a problem so i renamed and fix them

In [1]:
merged_iq_edu = pd.merge(
    avg_education[["Province Name", "Average Education Duration"]],
    avg_iq[["Province Name", "Average IQ"]],
    on="Province Name")

merged_iq_edu["Province Name"] = merged_iq_edu["Province Name"].apply(lambda x: unidecode(x))

merged_all_election = pd.merge(
    merged_iq_edu,
        election_result_percentages,
    on="Province Name")

NameError: name 'pd' is not defined

It's time to merge our three dataset

In [12]:
merged_all_election[["Average Education Duration","Average IQ","RECEP TAYYİP ERDOĞAN Percent"]].corr()

,Average Education Duration,Average IQ,RECEP TAYYİP ERDOĞAN Percent
Average Education Duration,1.000000,0.542144,-0.144464
Average IQ,0.542144,1.000000,-0.034847
RECEP TAYYİP ERDOĞAN Percent,-0.144464,-0.034847,1.000000


In [19]:
merged_all_election.corr().iloc[:2]

,Average Education Duration,Average IQ,RECEP TAYYİP ERDOĞAN Percent,MUHARREM İNCE Percent,KEMAL KILIÇDAROĞLU Percent,SİNAN OĞAN Percent
Average Education Duration,1.000000,0.542144,-0.144464,-0.025684,0.098162,0.306956
Average IQ,0.542144,1.000000,-0.034847,0.298690,-0.017048,0.366433


	•	The duration of education and average IQ seem to be related to the vote rates of some candidates (e.g., Sinan Oğan), but these relationships are not strong.
    
	•	Weak negative correlations are observed between Recep Tayyip Erdoğan’s vote rate and both the duration of education and IQ.